In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [3]:
# Blackjack Simulation Parameters
num_hands = 1000
num_simulations = 100

In [4]:
# Blackjack Probabilities (simplified - adjust for more accuracy)
# Example: Probability of winning a hand (adjust these)
win_prob = 0.49  # Example: Slightly less than 50% due to house edge
push_prob = 0.08  # Example: Probability of a push (tie)
loss_prob = 1 - win_prob - push_prob

In [ ]:
def simulate_coin_toss():
    random_number = np.random.rand()

    if random_number < 0.8:
        return "Heads"
    else:
        return "Tails"

In [30]:


b = 3/5
win = 0

initial_balance = 100
flips = 100

def get_kelly(b, p):
    return (b*p - (1-p))/b

for i in range(flips):
    amount = get_kelly(b, 0.7) * initial_balance
    outcome = simulate_coin_toss()    
    # initial_balance -= amount
    if outcome == "Heads":
        initial_balance += b * amount
    else:
        initial_balance -= amount
    if initial_balance <= 0:
        break
    print(initial_balance)



80.00000000000001
89.60000000000001
100.352
112.39424
125.88154879999999
140.98733465599997
157.90581481471997
126.324651851776
101.05972148142081
113.1868880591913
126.76931462629426
141.98163238144957
159.0194282672235
127.21554261377882
142.48140772743227
159.57917665472414
178.72867785329103
142.98294228263285
160.14089535654878
179.35780279933462
200.88073913525477
224.98642783148534
251.98479917126357
282.2229750718152
316.089732080433
354.02049993008495
396.50295992169515
444.0833151122985
355.2666520898389
284.21332167187114
318.3189202724957
254.65513621799658
285.21375256415615
319.4394028718549
255.55152229748396
286.21770497318204
320.56382956996384
359.0314891183595
402.1152678125626
321.69221425005014
360.2952799600561
403.5307135552628
451.95439918189436
361.56351934551554
404.9511416669774
453.54527866701466
362.8362229336118
406.37656968564517
455.14175804792256
364.1134064383381
407.80701521093863
456.74385703625126
365.3950856290011
409.2424959044812
327.393996723585

In [ ]:
def simulate_dice_roll():
    random_number = np.random.rand(0, 1)

    if random_number < 0.5:
        return 1
    else:
        # return a random number between 2 and 6 with a prob of 10% each
        return np.random.choice([2, 3, 4, 5, 6], p=[0.1, 0.1, 0.1, 0.1, 0.1])        

In [5]:
# Kelly Criterion Calculation
def kelly_fraction(win_prob, loss_prob):
    if win_prob <= loss_prob: # Avoid negative Kelly fraction.
        return 0.0
    return (win_prob - loss_prob) / (1.0) # Simplified for 1:1 payout

# Fractional Kelly
def fractional_kelly(win_prob, loss_prob, fraction):
    k = kelly_fraction(win_prob, loss_prob)
    return k * fraction

# Simulation Function
def run_simulation(bet_fraction, num_hands, num_simulations):
    final_balances = []
    for _ in range(num_simulations):
        balance = 1.0  # Initial bankroll (normalized to 1)
        for _ in range(num_hands):
            if np.random.rand() < win_prob:
                balance *= (1 + bet_fraction)
            elif np.random.rand() < win_prob + push_prob:
                pass # No change in balance for a push
            else:
                balance *= (1 - bet_fraction)
        final_balances.append(balance)
    return final_balances




In [6]:
# Interactive Widgets
win_prob_slider = widgets.FloatSlider(value=win_prob, min=0.3, max=0.6, step=0.01, description='Win Prob:')
fraction_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Kelly Fraction:')
num_hands_slider = widgets.IntSlider(value=num_hands, min=100, max=5000, step=100, description='Num Hands:')
num_sim_slider = widgets.IntSlider(value=num_simulations, min=100, max=1000, step=100, description='Num Sims:')


def update_plot(win_prob, fraction, num_hands, num_simulations):
    loss_prob = 1 - win_prob - push_prob  # Update loss probability
    k_optimal = kelly_fraction(win_prob, loss_prob)
    frac_kelly = fractional_kelly(win_prob, loss_prob, fraction)
    
    # Run simulations
    full_kelly_balances = run_simulation(k_optimal, num_hands, num_simulations)
    frac_kelly_balances = run_simulation(frac_kelly, num_hands, num_simulations)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.hist(full_kelly_balances, bins=50, alpha=0.5, label='Full Kelly')
    plt.hist(frac_kelly_balances, bins=50, alpha=0.5, label='Fractional Kelly')
    plt.xlabel('Final Bankroll')
    plt.ylabel('Frequency')
    plt.title(f'Blackjack Simulation (Win Prob: {win_prob:.2f}, Kelly Fraction: {fraction:.2f})')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    print(f"Optimal Kelly Fraction: {k_optimal:.4f}")
    print(f"Used Kelly Fraction: {frac_kelly:.4f}")
    print(f"Average Full Kelly Return: {np.mean(full_kelly_balances):.4f}")
    print(f"Average Fractional Kelly Return: {np.mean(frac_kelly_balances):.4f}")
    print(f"Median Full Kelly Return: {np.median(full_kelly_balances):.4f}")
    print(f"Median Fractional Kelly Return: {np.median(frac_kelly_balances):.4f}")


interactive_plot = widgets.interactive_output(update_plot, 
                                                 controls={'win_prob': win_prob_slider,
                                                           'fraction': fraction_slider,
                                                           'num_hands': num_hands_slider,
                                                           'num_simulations': num_sim_slider})

display(win_prob_slider, fraction_slider, num_hands_slider, num_sim_slider, interactive_plot)

FloatSlider(value=0.49, description='Win Prob:', max=0.6, min=0.3, step=0.01)

FloatSlider(value=0.5, description='Kelly Fraction:', max=1.0, step=0.05)

IntSlider(value=1000, description='Num Hands:', max=5000, min=100, step=100)

IntSlider(value=100, description='Num Sims:', max=1000, min=100, step=100)

Output()